# Ранжирование

Будем ранжировать разные данные с помощью XGBoost. <br>
В XGBoost используется алгоритм LambdaMART, который осуществляет pairwise ранжирование.

## Пример

Взят [отсюда](https://github.com/dmlc/xgboost/blob/master/demo/rank/rank_sklearn.py)

In [3]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import ndcg_score

Данные: датасет LETOR 4.0 MQ2008 - бенчмарк для ранжирования.

Данные представлены так:
* один объект - это запрос, один документ к нему (набор каких-то признаков) и одна метка релевантности (target)
* соответственно, для одного и того же запроса может быть несколько объектов
* информация, что, например, какие-то пять объектов относятся к одному запросу, содержится в отдельной структуре "groups" и передается в обучение

Читаем объекты и таргеты:

In [4]:
x_train, y_train = load_svmlight_file("mq2008.train")
x_valid, y_valid = load_svmlight_file("mq2008.vali")
x_test, y_test = load_svmlight_file("mq2008.test")

Посмотрим на данные:
* в обучении 9630 объектов
* 46 признаков
* релевантность оценивается по трехбалльной шкале

In [5]:
print(x_train.shape)
x_train[0].todense()

(9630, 46)


matrix([[0.007477, 0.      , 1.      , 0.      , 0.00747 , 0.      ,
         0.      , 0.      , 0.      , 0.      , 0.471076, 0.      ,
         1.      , 0.      , 0.477541, 0.00512 , 0.      , 0.571429,
         0.      , 0.004806, 0.768561, 0.727734, 0.716277, 0.582061,
         0.      , 0.      , 0.      , 0.      , 0.780495, 0.962382,
         0.999274, 0.961524, 0.      , 0.      , 0.      , 0.      ,
         0.797056, 0.697327, 0.721953, 0.582568, 0.      , 0.      ,
         0.      , 0.      , 0.      , 0.007042]])

In [6]:
set(y_train)

{0.0, 1.0, 2.0}

Читаем информацию о группах:

In [7]:
group_train = []
with open("mq2008.train.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_train.append(int(line.split("\n")[0]))

group_valid = []
with open("mq2008.vali.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_valid.append(int(line.split("\n")[0]))

group_test = []
with open("mq2008.test.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_test.append(int(line.split("\n")[0]))

Как устроена информация о группах:
* количество групп отражает информацию о количестве запросов
* каждое число обозначает количество последовательных объектов, которые в эту группу объединяются
* из предыдущего пункта следует, что в X объекты нельзя перемешивать
* если просуммировать все числа в списке групп, получим число объектов из X

Для чего нужны группы? <br>
Для того, чтобы в обучении не сравнивать доки из разных групп (разных запросов) между собой.

In [8]:
print(len(group_train), sum(group_train))
group_train[:10]

471 9630


[8, 8, 8, 8, 8, 16, 8, 118, 16, 8]

Обучаем модель. <br>
С помощью `eval_set` можем контролировать обучение, но это необязательный параметр, можно обучить и без валидации. <br>
В параметре `objective` можно задать три опции: `rank:ndcg`, `rank:pairwise`, `rank:map`. `ndcg` и `map` регулияруют попарный лосс с помощью подсчета соответствующих метрик.

In [9]:
params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 4}

model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, group_train, verbose=True,
          eval_set=[(x_valid, y_valid)], eval_group=[group_valid])

[0]	eval_0-map:0.707814
[1]	eval_0-map:0.709222
[2]	eval_0-map:0.7082
[3]	eval_0-map:0.714009


XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, gamma=1.0, learning_rate=0.1,
          max_delta_step=0, max_depth=6, min_child_weight=0.1, missing=None,
          n_estimators=4, n_jobs=-1, nthread=None, objective='rank:ndcg',
          random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
          seed=None, silent=None, subsample=1, verbosity=1)

Получим предсказание на тестовом сете:

In [10]:
pred = model.predict(x_test)

In [11]:
#pred

Посчитаем качество:

In [12]:
start_idx = 0
grouped_pred = []
grouped_target = []

for group_n in group_test:
    grouped_pred.append(pred[start_idx:start_idx+group_n])
    grouped_target.append(y_test[start_idx:start_idx+group_n])
    start_idx += group_n

In [13]:
np.mean([
    ndcg_score([grouped_target[i]], [grouped_pred[i]])
    for i in range(len(grouped_target))
])

0.49836693699277224

# Семинар и дз
Сделать и улучшить любую ML-модель на ваших проектных данных (просто клф, бленд, ранжирование, что-то что вы придумали сами...), используя любые признаки, какие захотите. Оцениваться будут:
* факт выполнения задания :)
* корректность кода (чтобы код не падал) и отсутствие логических ошибок (e.g. затестили на трейне)
* итеративность улучшения (например взяли один сет признаков, показали качество; потом добавили / подкрутили / использовали другую модель, показали качество...)
* креативность признаков
* аккуратность ноутбука

Дедлайн: 15 октября

In [14]:
import re
import pandas as pd
import unicodedata
from sklearn.model_selection import train_test_split

answers = pd.read_excel('answers_base.xlsx')
queries = pd.read_excel('queries_base.xlsx')

In [15]:
#new_seventy = queries[:int(len(queries)*0.7)]
#other_seventy = queries[1609:]#:-int(len(queries)*0.7)]
train, test = train_test_split(queries, test_size=0.3)

In [16]:
new_train = pd.DataFrame()
new_train['Текст вопроса'] = pd.concat([train['Текст вопроса'],answers['Текст вопросов']],ignore_index = True)
new_train['Номер связки\n'] = pd.concat([train['Номер связки\n'], answers['Номер связки']], ignore_index=True)
len(new_train)

1652

In [17]:
new_train

,Текст вопроса,Номер связки\n
0,"Добрый день, подскажите , что мне делать, если...",6.0
1,Добрый День!\n\nУ меня следующий вопрос.\n\n2 ...,308.0
2,"\nДобрый день!\t\nПодскажите, пожалуйста, тр...",37.0
3,"Получен тест на ковид,по прибытию из другой ст...",308.0
4,"Здравствуйте! У нас вопрос, 30.08.2000 приехал...",308.0
...,...,...
1647,Платные тесты на ковид?\nГде сдать тест на ков...,135.0
1648,"Мне 65 лет, куда обращаться, если работодатель...",5.0
1649,Куда жаловаться (обращаться) если я вижу наруш...,3.0
1650,Рекомендации для кафе и ресторанов?\nРекоменда...,45.0


In [18]:
#answers
from sklearn.feature_extraction.text import TfidfVectorizer

count_vect = TfidfVectorizer()

X_train = count_vect.fit_transform(new_train['Текст вопроса'].values.astype('U'))
y_train = new_train['Номер связки\n']

In [19]:
#test
X_test = count_vect.transform(test['Текст вопроса'].values.astype('U'))
y_test = test['Номер связки\n']

In [20]:
X_train.shape

(1652, 12551)

In [21]:
X_test.shape

(690, 12551)

In [22]:
#np.any(np.isnan(train))

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
from sklearn.metrics import accuracy_score

#X_train = X_train.fillna(' ')#reset_index()
y_train = y_train.fillna(0)#reset_index()
y_test = y_test.fillna(0)
#clf = MultinomialNB().fit(X_train, y_train)


#from sklearn.linear_model import LogisticRegression
#clf = LogisticRegression().fit(X_train, y_train)
#predicted = clf.predict(X_test)
#acc = accuracy_score(y_test, predicted)
#print('acc={0:1.4f}'.format(acc))

In [24]:
#clf = MultinomialNB().fit(X_train, y_train)
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
predicted = clf.predict(X_test)
acc = accuracy_score(y_test, predicted)
print('acc={0:1.4f}'.format(acc))

acc=0.5159


In [25]:
#print(len(predicted))
#print(len(test))

#['Predicted']


In [26]:
test['Predicted'] = predicted

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
#test

In [28]:
#print(len(new_train['Номер связки\n']), sum(new_train['Номер связки\n']))

In [29]:
#for tt in train['Номер связки\n']:
#  print(type(tt))
len(train['Номер связки\n'])

1609

In [30]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(score_func=chi2, k=10)

In [31]:
fit = test.fit(X_train, y_train)

In [32]:
np.set_printoptions(precision=3)

In [33]:
print(fit.scores_)

[1.237 0.749 1.21  ... 2.089 0.362 1.259]


In [34]:
features = fit.transform(X_train)

In [35]:
print(features) # Признаки через Хи-Квадрат

  (146, 0)	1.0
  (269, 8)	0.14079518638989102
  (270, 6)	0.0795228515138609
  (451, 0)	1.0
  (652, 2)	1.0
  (911, 0)	1.0
  (948, 3)	0.0830711154380084
  (956, 5)	1.0
  (1610, 3)	0.2584604936540927
  (1610, 2)	0.4484103074300624
  (1611, 3)	0.6168068838049291
  (1618, 6)	0.5279982000415716
  (1624, 7)	0.5885006304613428
  (1624, 8)	0.5575805461937295
  (1628, 1)	0.4287378332784641
  (1644, 4)	0.418372230486154
  (1644, 9)	0.418372230486154


In [36]:
accuracy_score(y_test,predicted)

0.5159420289855072